In [68]:
!pip install -q openai langchain beautifulsoup4 chroma chromadb tiktoken langchainhub crewai

In [65]:
from getpass import getpass
import os
os.environ['OPENAI_API_KEY'] = getpass()

 ········


In [73]:
print(openai.__version__)

NameError: name 'openai' is not defined

In [60]:
#Extract text from CDC website
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import time

#Send to GPT4 for cleanup
def clean_text_with_gpt4(text):
    """
    This function takes a string of text and uses GPT-4 to clean it up using the OpenAI ChatCompletion API.
    It handles large texts by breaking them into smaller chunks.
    :param text: String containing the text to be cleaned.
    :return: Cleaned text as a string.
    """
    cleaned_texts = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    chunks = text_splitter.split_documents(text)

    for chunk in chunks:
        try:
            print(chunk)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",  # Assuming using the latest GPT-4 model
                messages=[{"role": "system", "content": "You are a helpful assistant."},
                          {"role": "user", "content": f"Please clean up the following text:\n\n{chunk}"}]
            )
            print(response)
            cleaned_texts.append(response.choices[0].message.content.strip())
            #print(cleaned_texts)
            time.sleep(1)  # Delay to respect rate limits
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    return ' '.join(cleaned_texts)

file_name = "cleaned_texts.txt"
if os.path.exists(file_name):
    print(f"The file {file_name} already exists.")
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            content = file.read()
            #print("File content:\n")
            #print(content)
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    url = "https://www.cdc.gov"
    loader = RecursiveUrlLoader(
        url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs = loader.load()
    client = OpenAI()
    cleaned_texts = clean_text_with_gpt4(docs)
    with open(file_name, 'w', encoding='utf-8') as file:
        for text in cleaned_texts:
            file.write(text + "\n")  # Adding two newlines as a separator between texts
    print(f"Cleaned texts saved to {file_name}")
    cleaned_text = docs

page_content="Centers for Disease Control and Prevention\n \n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\n\nSkip directly to site content\nSkip directly to search\n\n\nEspaÃ±ol | \nOther Languages\n\n \n\n\n\n\nCenters for Disease Control and Prevention. CDC twenty four seven. Saving Lives, Protecting People\n\nCenters for Disease Control and Prevention. CDC twenty four seven. Saving Lives, Protecting People\n\n\n\n\n\n\n\n\n       \n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\nSubmit\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMissing Topic Title\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCLOSE\n\n\n\n\n\n\n\n\n\nCLOSE\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\nHealth Topics\n\n\nTravelers' Health\n\n\nOutbreaks\n\n\nData & Stats\n\n\nPublications\n\n\nAbout CDC\n\n\nCDC.GOV HOME\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\t\t\t\tCenters for Disease Control and Prevention\t\t\t\n\n\n\n\n\n\nSection Navigation\n\n\n\n\n\n\n\n\nCDC Home\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\

TypeError: 'NoneType' object is not iterable

In [85]:
print(bs4.__version__)
import langchain
print(langchain.__version__)
import openai
print(openai.__version__)
import chromadb
print(chromadb.__version__)
import langchain_community
print(langchain__community.__version__)
#with open(file_name, 'r') as file:
#    file_contents = file.read()
#cleaned_text = file_contents
#print(cleaned_text)

4.12.2
0.1.0
1.7.0
0.4.22


NameError: name 'langchain__community' is not defined

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter2.split_documents(cleaned_text)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
CDC_retriever = vectorstore.as_retriever()

ValueError: Expected IDs to be a non-empty list, got []

In [11]:
import zipfile
from langchain_community.document_loaders import DirectoryLoader

loader2 = DirectoryLoader('./EpiHiper-Schema-master', glob="**/*.py", show_progress=True)
abm_codes = loader2.load()

#code_splits = split_by_length_with_overlap(abm_codes)
vectorstore = Chroma.from_documents(documents=abm_codes, embedding=OpenAIEmbeddings())
ABM_retriever = vectorstore.as_retriever()






100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 46.05it/s]


In [15]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
import textwrap

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=.2)
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.0) #"gpt-4-1106-preview", "gpt-4-0314"

query_planner = Agent(
    role="Simulation Planner",
    goal="Plan the steps needed to parameterize an agent-based simulation from existing knowledge",
    backstory=textwrap.dedent("""
        You are an expert at identifying modeling parameters from code base that implements 
        an agent-based model and listing the model choices, parameters, and json config files 
        whose values need to be determined. You will break down each model choice, parameter, 
        and json config file into sub-questions such that the answer to each sub-question will 
        inform the value to be used in the agent-based simulation.
        Accept the user-question and determine if it requires sub-questions to either the
        CDC website which provides an official source of recent infectious disease outbreaks
        or Wikipedia for information about a geographical location, country, infectious agent
        characteristics, transmission dynamics, infection states, or other epidemiological
        modeling efforts.
        Your final answer MUST be a description of sub-questions that explain the best model
        choices, model parameters, and config files for an agent-based modeling code base.
    """),
    verbose=True,
    allow_delegation=False,
    tools=[],  ###
    llm=llm,
)


In [48]:
#from langchain import hub
#prompt = hub.pull("hwchase17/self-ask-with-search")
#print(f'{prompt.format(agent_scratchpad = "AGENTSCRATCHPAD", input = "INPUT")}')

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import DuckDuckGoSearchRun

CDC_retriever_tool = create_retriever_tool(
    CDC_retriever,
    "CDC_retriever_tool",
    """As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        
        Search for data related to outbreaks. For questions about outbreaks, use this tool to return 
        relevant data for answering questions about outbreaks""",
)

ABM_retriever_tool = create_retriever_tool(
    ABM_retriever,
    "ABM_retriever_tool",
    """As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        
        Search for data related to outbreaks. For questions about outbreaks, use this tool to return 
        relevant data for answering questions about outbreaks""",
)

web_search = DuckDuckGoSearchRun()

query_executor = Agent(
    role="""
        Agent Role: Information Searcher

        Primary Objectives:
        1. Utilize the CDC_retriever_tool to gather current outbreak information. This includes statistics, affected regions, and latest guidelines related to the outbreak.

        2. Employ the ABM_retriever_tool to access the codebase for agent-based modeling simulations. Extract relevant parameters and settings that are crucial for understanding the dynamics of the disease spread in the simulations.

        3. Conduct thorough internet searches using the web_search tool. Focus on disease-specific information such as modes of transmission, vectors involved, the role of asymptomatic infectious carriers, and insights from past modeling efforts.

        Key Responsibilities:
        - Ensure accurate and up-to-date information is retrieved from each tool.
        - Synthesize information from diverse sources to provide a comprehensive understanding of the disease and its impact.
        - Adhere to the principles of clarity and conciseness in reporting findings.
        """,
    goal="Information Searcher",
    backstory=textwrap.dedent("""
        Accept list of sub-questions from the query_planner agent and perform
        the necessary searches to answer the questions.
        Perform the tasks in the order given and report the result out.
        Your final answer MUST be a correct response to the original user-query.
    """),
    verbose=True,
    llm=llm,
    # tools=[SqlTools.do_sql_query, RagTools.do_rag_query],
    tools=[CDC_retriever_tool, ABM_retriever_tool, web_search],
    allow_delegation=True,
)

In [50]:
user_query = "Model the current flavivirus outbreak using an agent based model"

task0 = Task(
    description=textwrap.dedent(f"""
        Your task is to plan out the necessary steps in order to fulfull the user task and 
        create smaller prompts for other co-workers to follow. The result of this task should
        be an organized list of questions to be answered in order to fulfill the user request.
            {user_query}
    """),
    agent=query_planner
)

task1 = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of queries from the previous task. Your task is to update these
        queries based on the latest information from the CDC database using the CDC_retriever_tool
        tool and from a web search using the web_search tool in order to either make the queries 
        more specific or to provide the answers so they can be passed on to the next query.
        The information gathered should include key statistics such as number of cases, affected 
        regions, and any new guidelines issued. 
        Prepare the necessary sub-queries or search queries to handle this user-query:
            {user_query}
        You should use the CDC_retriever_tool and web_search to access the relevant data.
    """),
    agent=query_executor
)

task2 = Task(
    description=textwrap.dedent(f"""
        You will receive a set of queries from the previous task. 
        Use the ABM_retriever_tool to query the agent-based modeling codebase for parameters 
        and settings related to disease spread simulation. Report the parameters in the format
        required for input into the code base that is accessible from the ABM_retriever_tool.
        Your final answer must be a list of parameters and values or information that can be
        used to estimate those values in the appropriate format:
            {user_query}
    """),
    agent=query_executor
)

task3 = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of information from the previous task. Based on the information,
        your task is to populate the parameter keys and values into the final input format
        required for running the agent-based model.
        Your findings should directly support the original user-query:
            {user_query}
    """),
    agent=query_executor
)

crew = Crew(
    agents=[query_planner, query_executor],
    tasks=[task0, task1, task2, task3],
    verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result = crew.kickoff()

print("######################")
print(result)

Working Agent: Simulation Planner
Starting Task: 
Your task is to plan out the necessary steps in order to fulfull the user task and 
create smaller prompts for other co-workers to follow. The result of this task should
be an organized list of questions to be answered in order to fulfill the user request.
    Model the current flavivirus outbreak using an agent based model



> Entering new CrewAgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: To model the current flavivirus outbreak using an agent-based model, we need to gather specific information to accurately parameterize the simulation. Here is a breakdown of the model choices, parameters, and config files that need to be determined, along with the sub-questions that will inform their values:

1. Model Choice: Type of Agent-Based Model
   - What is the scale of the outbreak (local, regional, national, international)?
   - What is the primary mode of transmission for the flavivirus in question?
   - What

In [44]:
CDC_retriever_tool

Tool(name='Intermediate Answer', description="As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. \n\n        You always follow these guidelines:\n\n        -If the answer isn't available within the context, state that fact\n        -Otherwise, answer to your best capability, refering to source of documents provided\n        -Only use examples if explicitly requested\n        -Do not introduce examples outside of the context\n        -Do not answer if context is absent\n        -Limit responses to three or four sentences for clarity and conciseness\n        \n        Search for data related to outbreaks. For questions about outbreaks, use this tool to return \n        relevant data for answering questions about outbreaks", args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vecto

In [45]:
web_search

DuckDuckGoSearchRun()